In [9]:
import pandas as pd



In [10]:
data = pd.read_csv('data/fav_players.csv')
data.head()



,firstName,lastName,pos1,pos2,price,average,last3Avg,projected,include
0,Max,Gawn,Ruck,NaN,$944k,123.0,117.0,106.0,1
1,Zach,Merrett,Midfielder,NaN,$884k,115.0,116.0,111.0,1
2,Jake,Lloyd,Defender,NaN,$873k,114.0,113.0,104.0,1
3,Brodie,Grundy,Ruck,NaN,$867k,113.0,107.0,109.0,1
4,Matt,Crouch,Midfielder,NaN,$844k,110.0,122.0,110.0,1


In [18]:
# fantasy team parameters
salary_cap = 13000 # thousands of dollars
team_count = {
    "total": 30,
    "on_field": 22,
    "bench": 8,
    "defs": {
        "on_field": 6,
        "bench": 2
    },
    "mids": {
        "on_field": 8,
        "bench": 2
    },
    "rucks": {
        "on_field": 2,
        "bench": 1
    },
    "fwds": {
        "on_field": 6,
        "bench": 2
    }
}
print(team_count['fwds'])

{'on_field': 6, 'bench': 2}
